In [ ]:

# Question 1
import numpy as np
from scipy . stats import multivariate_normal as mvn
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import random
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt


def data(n):
  label = np.zeros((1,n))
  for i in range(n):
    label[0,i]= np.random.choice(np.arange(4),p=prior)
    
  x = np.zeros ((features , n))

  for index in range (n) :
    if label [0,index] == 0:
      x [:,index] = mvn(m0.reshape(3,),C0).rvs(1)
    elif label[0,index]==1:
      x [:,index] = mvn(m1.reshape(3,),C1).rvs(1) 
    elif label[0,index]==2:
      x [:,index] = mvn(m2.reshape(3,),C2).rvs(1) 
    else:
      x [:,index] = mvn(m3.reshape(3,),C3).rvs(1) 

  return x,label # return Data_concat
  
# theoretical optimal classifier
def theoretical_classifier(sample_type):
  data_opt,label_opt=data(sample_type) # sample_type=samples[6] for the test dataset
  lik=np.zeros((num_class,data_opt.shape[1]))
  tot_lik=np.zeros((num_class,data_opt.shape[1]))
  loss_mat=np.array([[0,1,1,1],[1,0,1,1],[1,1,0,1],[1,1,1,0]])
  for label in range(num_class):
    samp=mvn.pdf(data_opt.T,mean=mu_vector[label],cov=sigma_vector[label])
    lik[label]=samp
  prod=np.matmul(prior.reshape(1,num_class),lik)

  for label in range(num_class):
    tot_lik[label]=prod

  temp = prior.reshape(-1, 1)*lik/tot_lik 
  risk_mat = np.matmul(loss_mat, temp)

  label_sel=np.argmin(risk_mat,axis=0)

  correct_samp=[]
  incorrect_samp=[]

  for label in range(num_class):
    lbl=(label_opt==label)
    corr_lbl=((lbl)*(label_sel==lbl)).astype('int')
    incorr_lbl=((lbl)*(label_sel!=lbl)).astype('int')
    correct_=np.where(corr_lbl==1)[0]
    incorrect_=np.where(incorr_lbl==1)[0]
    
    correct_samp.append(correct_)
    incorrect_samp.append(incorrect_)

  p_err=1.0*np.sum((label_sel!=label_opt).astype('int'))/data_opt.shape[1]
  print(f'Theoretical Minimum Probability of Error:{p_err.round(4)}')
  
def split_data(data, folds):
  data_split = []
  random.seed(1)
  data_temp = list(data)
  fold_size = int(len(data) / folds)
  for i in range(folds):
    fold = list()
    while len(fold) < fold_size:
      rand_index = random.randrange(len(data_temp))
      fold.append(data_temp.pop(rand_index))
    data_split.append(fold)
  return data_split

def test_train_split(folds,i):
  test=np.array(folds.pop(i)) 
  train=np.array(folds)
  xtest=test[:,0:3]
  ytest=test[:,3]
  xtrain=train[:,:,0:3].reshape(-1,3)
  ytrain=train[:,:,3].reshape(-1)

  return xtrain,ytrain,xtest,ytest

def perceptrons_sel(sample_type):

  error_mat=np.zeros((len(percept_list),len(activations)))
  accuracy_mat=np.zeros((len(percept_list),len(activations)))

  weights_perceptron_layer1=[]
  weights_perceptron_layer2=[]
  data_train,label_train=data(sample_type)
  data_concat=np.hstack((data_train.T,label_train.T))

  for activ in range(len(activations)):

    activ_weight1=[]
    activ_weight2=[]

    for idx,percpt in enumerate(percept_list):
      err_fold=[]
      acc_fold=[]
      
      for i in range(num_folds):
        X_train,Y_train,X_test,Y_test=test_train_split(split_data(data_concat,num_folds),i)
        input_shape = (features,)    
        Y_train_encod=to_categorical(Y_train,num_class)  
        model = tf.keras.models.Sequential()

        model.add(Dense(percpt,kernel_initializer='random_uniform', input_shape=input_shape, activation=activations[activ]))
        model.add(Dense(num_class, kernel_initializer='random_uniform', activation='softmax'))

        model.compile( optimizer='Adam',loss='categorical_crossentropy', metrics=['accuracy'])
        model.fit(X_train,Y_train_encod, epochs=10, batch_size=10, verbose=0) 
        temp_y=model.predict(X_test) 
        y_pred=np.argmax(temp_y, axis=1)
        tmt=to_categorical(Y_test,num_class)
        y_test=np.argmax(tmt, axis=1)
        
        cm = confusion_matrix(y_test, y_pred,labels=[0.0,1.0,2.0,3.0])
        temp=(cm[0,0]+cm[1,1]+cm[2,2]+cm[3,3])/cm.sum()
        err_fold.append(1-temp)
        acc_fold.append(temp)

      error_mat[idx][activ]=(np.mean(err_fold)) # choose the perceptron based on the min of error
      accuracy_mat[idx][activ]=(np.mean(acc_fold))

      activ_weight1.append((model.layers[0].get_weights())[0])
      activ_weight2.append((model.layers[1].get_weights())[0])
    weights_perceptron_layer1.append(activ_weight1)
    weights_perceptron_layer2.append(activ_weight2)

  # test the new data 
   # find the perceptron with minimum error or loss
  opt_percerptron=np.where(error_mat==error_mat.flat[np.argmin(error_mat)])
  optimal_perceptron=percept_list[int(opt_percerptron[0])]
  optimal_activation=activations[int(opt_percerptron[1])]

  print(f'activation:{optimal_activation}')
  print(f'optimal number of neurons:{optimal_perceptron}')

  init1=  tf.constant_initializer(weights_perceptron_layer1[int(opt_percerptron[1])][int(opt_percerptron[0])])
  init2=  tf.constant_initializer(weights_perceptron_layer2[int(opt_percerptron[1])][int(opt_percerptron[0])])


  data_test,label_test=data(samples[6]) # test dataset
  X_train_=data_train.T
  X_test_=data_test.T

  Y_train_ = to_categorical(label_train.T,num_class)
  Y_test_ = to_categorical(label_test.T, num_class)

  X_train_ = X_train_.reshape(X_train_.shape[0], features)
  X_test_ = X_test_.reshape(X_test_.shape[0], features)

  input_shape = (features,)
  print(f'Feature shape: {input_shape}')

  model = tf.keras.Sequential()
  model.add(Dense(optimal_perceptron,kernel_initializer=init1, input_shape=input_shape, activation=optimal_activation))
  model.add(Dense(num_class, kernel_initializer=init2,activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.fit(X_train_, Y_train_, epochs=10, batch_size=32, verbose=1) 

 
  test_results = model.evaluate(X_test_, Y_test_, verbose=1)
  print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}')

  y_pred_=model.predict(X_test_) 
  y_pred_=np.argmax(y_pred_, axis=1)
  y_test_=np.argmax(Y_test_, axis=1)
  cm = confusion_matrix(y_test_, y_pred_)
  normalized_cm=cm / cm.astype(float).sum(axis=1)
  error = (1- (cm[0,0]+cm[1,1]+cm[2,2]+cm[3,3])/cm.sum())

  print(f'minimum probability of error:{error}')
  print(f'confusion matrix:{cm}')
  print(f'Normalized confusion matrix:{normalized_cm}')
  
  return data_train,label_train,error_mat

def plot_results(sample_type):
  dataset_train,labels_training,error_matt=perceptrons_sel(sample_type)
  #plot actual data distribution
  actual_data=np.array(dataset_train.T)
  actual_label=np.array(labels_training.T)

  x0=[actual_data[i] for i in range(sample_type) if actual_label[i]==0]
  x1=[actual_data[i] for i in range(sample_type) if actual_label[i]==1]
  x2=[actual_data[i] for i in range(sample_type) if actual_label[i]==2]
  x3=[actual_data[i] for i in range(sample_type) if actual_label[i]==3]

  fig = plt.figure(figsize = (10, 7))
  ax = plt.axes(projection ="3d")

  ax.scatter3D((np.array(x0))[:,0],(np.array(x0))[:,1],(np.array(x0))[:,2])
  ax.scatter3D((np.array(x1))[:,0],(np.array(x1))[:,1],(np.array(x1))[:,2])
  ax.scatter3D((np.array(x2))[:,0],(np.array(x2))[:,1],(np.array(x2))[:,2])
  ax.scatter3D((np.array(x3))[:,0],(np.array(x3))[:,1],(np.array(x3))[:,2])
  plt.legend(['class0','class 1','class 2','class 3'])
  plt.title('Actual Data Distribution')
  plt.show

  #plot min error vs number of perceptrons

  tt=np.array(percept_list)
  bar1=error_matt[:,0]
  bar2=error_matt[:,1]
  bar3=error_matt[:,2]

  fig = plt.figure(figsize = (10, 7))
  ax2 = plt.axes()
  ax2.scatter(tt,bar1)
  ax2.plot(tt,bar1)

  ax2.scatter(tt,bar2)
  ax2.plot(tt,bar2) 

  ax2.scatter(tt,bar3)
  ax2.plot(tt,bar3)

  plt.legend(activations)
  plt.xlabel('Number of perceptrons') 
  plt.ylabel('Average minimum Probability of Error') 
  plt.title('Minimum Error for different number of neurons')
  plt.show()


def main():
  plot_results(samples[0])
  theoretical_classifier(samples[6])
if __name__ == "__main__":
  prior=np.array([0.25,0.25,0.25,0.25])
  features=3
  num_class=4
  samples = [100,200,500,1000,2000,5000,100000]
  m0 = np.array([20, 0, 0])
  m1 = np.array([0, 5, 10])
  m2 = np.array([5, 0, 15])
  m3 = np.array([20, 15,5])

  mu_vector = [m0, m1, m2, m3]

  C0 = np.array([[10, 0, 10], [5, 40, 0], [5, 0, 20]])
  C1 = np.array([[40, 0, 0], [0, 25, 0], [0, 0, 10]])
  C2 = np.array([[10, 0, 0], [0, 20, 0], [0, 0, 30]])
  C3 = np.array([[15, 0, 0], [0, 20, 10], [0, 0, 25]])
  sigma_vector = [C0, C1, C2, C3]

  activations=['relu','elu','sigmoid']
  num_folds=10
  percept_list=np.logspace(1,3,num = 20,endpoint = True,base = 5,dtype = int)
  percept_list=percept_list.tolist()
  main()


In [ ]:
# Question 2
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import cross_val_score
from scipy . stats import multivariate_normal as mvn


def data(n):
  label = np.zeros((1,n))

  for i in range(n):
    label[0,i]=np.random.choice(np.arange(4),p=prior)

  x = np.zeros ((features , n))
  for i in range (n) :
    if label[0,i]==0:
      x [:,i] = mvn(m0.reshape(2,),C0).rvs(1)
    elif label[0,i]==1:
      x [:,i] = mvn(m1.reshape(2,),C1).rvs(1)
    elif label[0,i]==2:
      x [:,i] = mvn(m2.reshape(2,),C2).rvs(1)
    else:
      x [:,i] = mvn(m3.reshape(2,),C3).rvs(1) 
 
  return x,label 

def model_order(sample_type):# model order selection
  pos_log=[]
  rates=[]
  for k in range(num_expt):
    Data_train,label=data(sample_type)
    gmm_avg=np.zeros((len(gmm_list)))
    gmm_var=np.zeros((len(gmm_list)))
    temp=[]
    for idx,i in enumerate(gmm_list):
      gmm=GaussianMixture(i,covariance_type='full',random_state=None)
      scor=cross_val_score(gmm,Data_train.T,cv=num_fold) 
      avg_scor=np.mean(scor)
      var_scor=np.std(scor)

      gmm_avg[idx]=avg_scor
      gmm_var[idx]=var_scor
      
      gmm.fit(Data_train.T)
      temp.append(avg_scor)
      
    model_sel=np.argmax(gmm_avg) + 1
    model_sel_list[k]=int(model_sel)
    pos_log.append(temp)

# calculating the rates
  for i in range(len(gmm_list)):
    temp_list=(list(model_sel_list).count(i+1))/num_expt
    rates.append(round(temp_list,2)) 

  print(f'rates_{sample_type}:{rates}' )
  return model_sel_list,pos_log

def plot_results(sample_type):

  #plot the actual data distribution
  dataset_train,labels_training=data(sample_type) 
  actual_data=np.array(dataset_train.T)
  actual_label=np.array(labels_training.T)

  x0=[actual_data[i] for i in range(sample_type) if actual_label[i]==0]
  x1=[actual_data[i] for i in range(sample_type) if actual_label[i]==1]
  x2=[actual_data[i] for i in range(sample_type) if actual_label[i]==2]
  x3=[actual_data[i] for i in range(sample_type) if actual_label[i]==3]

  plt.scatter((np.array(x0))[:,0],(np.array(x0))[:,1])
  plt.scatter((np.array(x1))[:,0],(np.array(x1))[:,1])
  plt.scatter((np.array(x2))[:,0],(np.array(x2))[:,1])
  plt.scatter((np.array(x3))[:,0],(np.array(x3))[:,1])
  plt.legend(['class0','class 1','class 2','class 3'])
  plt.title('Actual Data Distribution')
  plt.show

#second plot
  selected_list,pos_list=model_order(sample_type)
  n_bins=len(gmm_list)
  fig,ax=plt.subplots(tight_layout=True)
  ax.set_xlim([1,6.5])
  ax.hist(selected_list,bins=n_bins,color='saddlebrown')
  plt.xlabel('model order')
  plt.ylabel('frequency of model order')
  plt.title('Frequency of model orders selected')
  plt.show()

#third plot
  vv=np.array(pos_list) # return the pos_log
  loss_=[np.mean(vv[:,i]) for i in range(vv.shape[1]) ]
  plt.plot(np.arange(1,len(gmm_list)+1),loss_,c='b', mfc='red',marker='o')
  plt.xlabel('model order')
  plt.ylabel(f'log likelihood')
  plt.title(f'log likelihood for dataset_{sample_type} under different GMM model orders',fontsize=12.)
  plt.show()

def main():
  plot_results(samples[0])
  plot_results(samples[1])
  plot_results(samples[2])
  plot_results(samples[3])
  
     
if __name__ == "__main__":

    prior=np.array([0.1,0.2,0.3,0.4])
    features=2
    num_class=4
    samples = [10,100,1000,10000]
    m0 = np.array([0,50])
    m1 = np.array([30,50])
    m2 = np.array([0,0])
    m3 = np.array([30, 0])

    mu_vector = [m0, m1, m2, m3]

    C0 = np.array([[10, 0], [10, 20]])
    C1 = np.array([[20, 30], [0, 10]])
    C2 = np.array([[20, 30], [0, 10]])
    C3 = np.array([[10, 20], [0, 40]]) 
    
    sigma_vector = [C0, C1, C2, C3]

    gmm_list=[1,2,3,4,5,6]
    num_expt=30
    num_fold=10
    model_sel_list=np.zeros((num_expt))
    main()

